In [1]:
import pandas as pd
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler
import pickle

In [2]:
current_year = 2023

In [3]:
market = Market()
spec = ADatabase("spec")
umod = UniversalModeler()

In [4]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [5]:
factors = [str(x) for x in range(14)]

In [6]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
    "y"
]

In [7]:
included_columns.extend(factors)

In [8]:
training_sets = []

In [9]:
for ticker in sp500["ticker"].unique():
    try:
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        ticker_data = prices[prices["ticker"]==ticker]
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
        for i in range(14):
            ticker_data[str(i)] = ticker_data["adjclose"].shift(i)
        ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=1)
        ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
        ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
        ticker_data["rolling14"] = ticker_data["adjclose"].rolling(window=14).mean()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data["y"] = ticker_data["adjclose"].shift(-1)
        ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
        ticker_data.dropna(inplace=True)
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except Exception as e:
        print(ticker,str(e))

DRE 'date'
FBHS 'date'


In [10]:
data = pd.concat(training_sets)

In [11]:
data = data.dropna()

In [12]:
spec.connect()
for modeler in [umod]:
    for year in tqdm(range(current_year,current_year+1)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - 4)].reset_index(drop=True)
        set_name = "models"
        stuff = modeler.recommend_model(training_slice,factors)
spec.disconnect()

  0%|                                                                                                                                                              | 0/1 [00:00<?, ?it/s]

Epoch 1/10
2408/2408 [==============================] - 2s 795us/step - loss: 884.3224 - mean_absolute_percentage_error: 8.1248
Epoch 2/10
2408/2408 [==============================] - 2s 819us/step - loss: 430.2755 - mean_absolute_percentage_error: 6.1410
Epoch 3/10
2408/2408 [==============================] - 2s 877us/step - loss: 389.5782 - mean_absolute_percentage_error: 5.7207
Epoch 4/10
2408/2408 [==============================] - 2s 825us/step - loss: 332.6267 - mean_absolute_percentage_error: 5.1339
Epoch 5/10
2408/2408 [==============================] - 2s 758us/step - loss: 270.5714 - mean_absolute_percentage_error: 4.8338
Epoch 6/10
2408/2408 [==============================] - 2s 749us/step - loss: 309.8250 - mean_absolute_percentage_error: 5.0748
Epoch 7/10
2408/2408 [==============================] - 2s 753us/step - loss: 330.2628 - mean_absolute_percentage_error: 4.9935
Epoch 8/10
2408/2408 [==============================] - 2s 761us/step - loss: 303.7166 - mean_absolute_p

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:28<00:00, 88.83s/it]


In [13]:
stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...metrics\mean_absolute_percentage_error
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-03 11:17:12         2522
metadata.json                                  2023-03-03 11:17:12           64
variables.h5                                   2023-03-03 11:17:12        84160


In [14]:
spec.connect()
models = spec.store("models",stuff)
spec.disconnect()